# Fermentation data import and export

## 1. Imports and directory definitions

In [1]:
import pandas as pd # pandas for tabular data manipulation
import numpy as np  # numpy for numerical operations
import glob
import os
from pandas import ExcelWriter
import openpyxl

data_folder = '../../../data/'
fieldlist_path = '../fieldlist/fieldlist_v1.xlsx'
od_cal_file = '../2021-10-04_OD_calibration_file.xlsx'

## 2. Experient name definition

In [2]:
# "../../../data/LIMS_data/DDB_PD_005_AMBR_LIMS.xlsx"
experiment = 'DDB_PD_005_AMBR'

## 3. Import data

### 3.1 Main cultures metadata

In [3]:
# get the path to experiment in the LIMS file folder
experiment_file = data_folder+'LIMS_data/'+experiment+'_LIMS.xlsx'
# import the data from the main culture sheet and drop any column that has only NaNs
meta_data = pd.read_excel(experiment_file, sheet_name='main culture').dropna(how='all', axis=1)
    
# cultures.index = range(0,len(cultures.index)) # updating the index of the dataframe
meta_data.rename(columns={'Sample ID': 'Parent Culture'}, inplace=True)
meta_data.sort_values('Reactor/Plate/Flask Number',inplace=True)
meta_data = meta_data.reset_index().drop('index',axis=1)
meta_data

,Parent Culture,Base Medium,Container ID (calculated),Container Type,Control?,Culture Type,Experiment,Experiment Annotation,growth_rate,Feed Medium #1,...,Minimum Aeration (slpm),Maximum Aeration (slpm),Minimum Stirring or Shaking Speed (rpm),Maximum Stirring or Shaking Speed (rpm),Aeration Gas Type,Final Seed OD,Volume to AddNone,Volume AddedNone,Volume UnitNone,Inoculate Date/Time
0,DDB_PD_005_AMBR_DDB7-002_R01_Main_,DDB_BM_002,R01,AMBR 250,No,Main,DDB_PD_005_AMBR,DDB Quorum Sensing,0.11,DDB_FM_002,...,0.1,0.23,1000,4000,Air,5.64250,3.544528,3.67,mL,2020-08-18 10:00:00
1,DDB_PD_005_AMBR_DDB7-002_R02_Main_,DDB_BM_002,R02,AMBR 250,No,Main,DDB_PD_005_AMBR,DDB Quorum Sensing,0.11,DDB_FM_002,...,0.1,0.23,1000,4000,Air,5.64250,3.544528,3.67,mL,2020-08-18 10:00:00
2,DDB_PD_005_AMBR_DDB12-001_R03_Main_,DDB_BM_002,R03,AMBR 250,No,Main,DDB_PD_005_AMBR,DDB Quorum Sensing,0.11,DDB_FM_002,...,0.1,0.23,1000,4000,Air,5.56750,3.592277,3.73,mL,2020-08-18 10:00:00
3,DDB_PD_005_AMBR_DDB12-001_R04_Main_,DDB_BM_002,R04,AMBR 250,No,Main,DDB_PD_005_AMBR,DDB Quorum Sensing,0.11,DDB_FM_002,...,0.1,0.23,1000,4000,Air,5.56750,3.592277,3.73,mL,2020-08-18 10:00:00
4,DDB_PD_005_AMBR_DDB15-001_R05_Main_,DDB_BM_002,R05,AMBR 250,No,Main,DDB_PD_005_AMBR,DDB Quorum Sensing,0.11,DDB_FM_002,...,0.1,0.23,1000,4000,Air,5.89250,3.394145,3.51,mL,2020-08-18 11:00:00
5,DDB_PD_005_AMBR_DDB15-001_R06_Main_,DDB_BM_002,R06,AMBR 250,No,Main,DDB_PD_005_AMBR,DDB Quorum Sensing,0.11,DDB_FM_002,...,0.1,0.23,1000,4000,Air,5.89250,3.394145,3.51,mL,2020-08-18 11:00:00
6,DDB_PD_005_AMBR_DDB16-001_R07_Main_,DDB_BM_002,R07,AMBR 250,No,Main,DDB_PD_005_AMBR,DDB Quorum Sensing,0.11,DDB_FM_002,...,0.1,0.23,1000,4000,Air,5.40525,3.700106,3.84,mL,2020-08-18 10:00:00
7,DDB_PD_005_AMBR_DDB16-001_R08_Main_,DDB_BM_002,R08,AMBR 250,No,Main,DDB_PD_005_AMBR,DDB Quorum Sensing,0.11,DDB_FM_002,...,0.1,0.23,1000,4000,Air,5.40525,3.700106,3.84,mL,2020-08-18 10:00:00


### 3.1 Online data

In [4]:
online_data_dir = 'online_data_interpolated/'
csv_prefix = '_online_stacked_2m.csv'

In [5]:
# import fieldlist
fieldlist = pd.read_excel(fieldlist_path)
# initialize an empty DataFrame with columns from the variables specified in the fieldlist
online_data = pd.DataFrame(columns=list(fieldlist.new_variable_name))

In [6]:
ambr_df = pd.read_csv(data_folder+ online_data_dir + str(experiment) + csv_prefix)

for column in list(online_data.columns): # iterate over the columns of the temporary dataframe. The columns are the variable names of the online data.
    # match the column name (new_variable_name) to the name given by ambr250. Then assign the ambr250 online data to the matching column in the temp_df
    online_data[column] = ambr_df[fieldlist[fieldlist.new_variable_name == column].AMBR_name.iloc[0]]

# adding a column with the experiment name
online_data['Experiment'] = experiment

# Converting the bioreactor naming format in the temporary ambr df
for i in range(1,10):
    ambr_df['Batch ID'] = ambr_df['Batch ID'].replace('Bioreactor '+str(i),'R0'+str(i))
for i in range(10,25):
    ambr_df['Batch ID'] = ambr_df['Batch ID'].replace('Bioreactor '+str(i),'R'+str(i))

# adding a column with the Reactor Number
online_data['Reactor/Plate/Flask Number'] = ambr_df['Batch ID'] # adding a column with the reactor ID
    
# dropping some unnecessary rows from the online data DataFrame:
online_data.dropna(axis=0, subset=['Inlet_air_O2'],inplace=True)
    
# updating index
online_data.index = range(0,len(online_data.index)) # updating the index of the dataframe
    
# Merging online data with cultures metadata
online_data = pd.merge(left=online_data,right=meta_data)
    
# phase annotation
feed_vol_list = online_data['Feed_volume'].tolist()
phase_list = ['feed' if i > 0.0 else 'batch' for i in feed_vol_list]
online_data['Phase'] = phase_list
online_data.head()

,Time,DO,pH,Stirrer_speed,Temperature,Pressure,Air_flow,Oxygen_flow,Inlet_air_O2,Offgas_O2,...,Maximum Aeration (slpm),Minimum Stirring or Shaking Speed (rpm),Maximum Stirring or Shaking Speed (rpm),Aeration Gas Type,Final Seed OD,Volume to AddNone,Volume AddedNone,Volume UnitNone,Inoculate Date/Time,Phase
0,0.000000,93.420257,6.588931,1000.863155,29.969999,2.625100,99.969935,0.0,20.95,20.665001,...,0.23,1000,4000,Air,5.6425,3.544528,3.67,mL,2020-08-18 10:00:00,batch
1,0.033333,92.407015,6.588931,999.000000,30.000000,0.124776,99.999790,0.0,20.95,20.662535,...,0.23,1000,4000,Air,5.6425,3.544528,3.67,mL,2020-08-18 10:00:00,batch
2,0.066667,91.897315,6.589056,1000.117931,30.011180,2.507053,99.987134,0.0,20.95,20.645826,...,0.23,1000,4000,Air,5.6425,3.544528,3.67,mL,2020-08-18 10:00:00,batch
3,0.100000,91.624246,6.588078,1004.455012,30.030001,2.344355,100.037407,0.0,20.95,20.652922,...,0.23,1000,4000,Air,5.6425,3.544528,3.67,mL,2020-08-18 10:00:00,batch
4,0.133333,91.619059,6.589123,1001.030660,30.030001,2.554748,99.945202,0.0,20.95,20.670846,...,0.23,1000,4000,Air,5.6425,3.544528,3.67,mL,2020-08-18 10:00:00,batch


### 3.2 Offline data

#### 3.2.1 Timepoint data

In [7]:
timepoint_cols = ['Entity','Experiment','Parent Culture','Reactor/Plate Number','Timepoint (#)','Timepoint (h)','Volume','OD']
file = data_folder+'/LIMS_data/'+experiment+'_LIMS.xlsx' # get the path to experimentin the LIMS file folder

timepoint_data = pd.read_excel(file, sheet_name='timepoint samples')[timepoint_cols] # importing timepoint data from one experiment into a temporary DFF

# renaming some of the columns in the dataframe and reindexing
timepoint_data.rename(columns={"Entity": "Timepoint Sample", "Reactor/Plate Number": "Reactor/Plate/Flask Number", "Volume": "Sample volume (mL)"},inplace=True)
timepoint_data = timepoint_data.reset_index().drop('index',axis=1)
timepoint_data.head()

,Timepoint Sample,Experiment,Parent Culture,Reactor/Plate/Flask Number,Timepoint (#),Timepoint (h),Sample volume (mL),OD
0,DDB_PD_005_AMBR_R01__S00,DDB_PD_005_AMBR,DDB_PD_005_AMBR_DDB7-002_R01_Main_,R01,S00,0.036495,1.0,0.256
1,DDB_PD_005_AMBR_R01__S01,DDB_PD_005_AMBR,DDB_PD_005_AMBR_DDB7-002_R01_Main_,R01,S01,3.047016,1.0,0.628
2,DDB_PD_005_AMBR_R01__S02,DDB_PD_005_AMBR,DDB_PD_005_AMBR_DDB7-002_R01_Main_,R01,S02,6.018670,2.5,1.800
3,DDB_PD_005_AMBR_R01__S03,DDB_PD_005_AMBR,DDB_PD_005_AMBR_DDB7-002_R01_Main_,R01,S03,9.016564,1.0,5.365
4,DDB_PD_005_AMBR_R01__S04,DDB_PD_005_AMBR,DDB_PD_005_AMBR_DDB7-002_R01_Main_,R01,S04,12.019295,1.0,14.650


#### 3.2.2 Analytical data

In [8]:
analytical_cols = ['Name', 'Analysis Type', 'Cellular Compartment', 'Dilution Factor','Experiment', 'Parent Culture', 'Sample Prep Date', 'Timepoint (h)', 'Timepoint Sample', 'compound_name', 'concentration','concentration_unit']
    
analytical_data = pd.DataFrame(columns=analytical_cols) # initializing an empty DF for timepoint_data
file = data_folder+'/LIMS_data/'+experiment+'_LIMS.xlsx' # get the path to experiment in the LIMS file folder

analytical_data = pd.read_excel(file, sheet_name='analytical samples')[analytical_cols] # importing timepoint data from one experiment into a temporary DF

# renaming one of the columns in the dataframe and reindexing
analytical_data.rename(columns={"Name": "Analytical Sample"},inplace=True)
analytical_data = analytical_data.reset_index().drop('index',axis=1)

# correcting for the dilution factor, then dropping the dilution factor
analytical_data['concentration'] = analytical_data['concentration'] * analytical_data['Dilution Factor']
analytical_data.drop(['Dilution Factor'], axis=1, inplace=True)
# removing any analytical data not obtained from a main culture sample
analytical_data = analytical_data[analytical_data['Parent Culture'].notnull()]
analytical_data.head()

,Analytical Sample,Analysis Type,Cellular Compartment,Experiment,Parent Culture,Sample Prep Date,Timepoint (h),Timepoint Sample,compound_name,concentration,concentration_unit
0,DDB_PD_005_AMBR_R01__S00_SOA_#1,SOA,Extracellular Region,DDB_PD_005_AMBR,DDB_PD_005_AMBR_DDB7-002_R01_Main_,2020-08-20,0.036495,DDB_PD_005_AMBR_R01__S00,succinic acid,0.000000,g/L
1,DDB_PD_005_AMBR_R01__S00_SOA_#1,SOA,Extracellular Region,DDB_PD_005_AMBR,DDB_PD_005_AMBR_DDB7-002_R01_Main_,2020-08-20,0.036495,DDB_PD_005_AMBR_R01__S00,ethanol,0.175717,g/L
2,DDB_PD_005_AMBR_R01__S00_SOA_#1,SOA,Extracellular Region,DDB_PD_005_AMBR,DDB_PD_005_AMBR_DDB7-002_R01_Main_,2020-08-20,0.036495,DDB_PD_005_AMBR_R01__S00,potassium dihydrogen phosphate,9.618414,g/L
3,DDB_PD_005_AMBR_R01__S00_SOA_#1,SOA,Extracellular Region,DDB_PD_005_AMBR,DDB_PD_005_AMBR_DDB7-002_R01_Main_,2020-08-20,0.036495,DDB_PD_005_AMBR_R01__S00,citric acid,0.000000,g/L
4,DDB_PD_005_AMBR_R01__S00_SOA_#1,SOA,Extracellular Region,DDB_PD_005_AMBR,DDB_PD_005_AMBR_DDB7-002_R01_Main_,2020-08-20,0.036495,DDB_PD_005_AMBR_R01__S00,D-glucose,21.867193,g/L


##### 3.2.2.1 Pivot analytical data

In [9]:
analytical_data['compound_name_and_unit'] = analytical_data['compound_name'] + ' [' + analytical_data['concentration_unit'] + ']'

analytical_data_pivot = analytical_data.pivot(index='Timepoint Sample', columns=['compound_name_and_unit'], values='concentration')
analytical_data_pivot['Timepoint Sample'] = analytical_data_pivot.index

analytical_data_pivot.index.name = None
analytical_data_pivot.columns.name = None
analytical_data_pivot = analytical_data_pivot.reset_index().drop('index',axis=1)

analytical_data_pivot.head()

,D-glucose [g/L],acetic acid [g/L],citric acid [g/L],ethanol [g/L],formic acid [g/L],lactate [g/L],potassium dihydrogen phosphate [g/L],pyruvic acid [g/L],succinic acid [g/L],Timepoint Sample
0,21.867193,0.000000,0.0,0.175717,0.0,0.000000,9.618414,0.000000,0.0,DDB_PD_005_AMBR_R01__S00
1,21.757897,0.000000,0.0,0.141002,0.0,0.000000,11.672140,0.000000,0.0,DDB_PD_005_AMBR_R01__S01
2,20.801498,0.129068,0.0,0.106812,0.0,0.000000,10.974623,0.000000,0.0,DDB_PD_005_AMBR_R01__S02
3,16.772116,0.362876,0.0,0.146651,0.0,0.580863,9.833429,0.098438,0.0,DDB_PD_005_AMBR_R01__S03
4,9.421337,0.568955,0.0,0.191405,0.0,1.082594,9.341226,0.308042,0.0,DDB_PD_005_AMBR_R01__S04


#### 3.2.3 Trace_metal_data

In [10]:
trace_metal_data = pd.read_excel(data_folder+'/LIMS_data/'+experiment+'_LIMS.xlsx', sheet_name='trace metal export')
trace_metal_data.rename(columns={'Name': 'Analytical Sample'}, inplace=True)

# timepoint_samples_temp = []
# for i in df_tm['Analytical Sample'].to_list():
#     timepoint_samples_temp.append(i.split('_TM')[0])

# df_tm['Timepoint Sample'] = timepoint_samples_temp
trace_metal_data['Timepoint Sample'] = trace_metal_data['Analytical Sample'].apply(lambda x: pd.Series(str(x).split("_TM")[0]))
trace_metal_data.head()

,Analytical Sample,compound_name,concentration,concentration_unit,Timepoint Sample


##### 3.2.2.1 Pivot trace metal data

In [11]:
trace_metal_data['compound_name_and_unit'] = trace_metal_data['compound_name'] + ' [' + trace_metal_data['concentration_unit'] + ']'

trace_metal_data_pivot = trace_metal_data.pivot(index='Timepoint Sample', columns=['compound_name_and_unit'], values='concentration')
trace_metal_data_pivot['Timepoint Sample'] = trace_metal_data_pivot.index

trace_metal_data_pivot.index.name = None
trace_metal_data_pivot.columns.name = None
trace_metal_data_pivot = trace_metal_data_pivot.reset_index().drop('index',axis=1)

trace_metal_data_pivot.head()

,Timepoint Sample


#### 3.2.4 Custom data

In [12]:
try:
    custom_data = pd.read_excel(data_folder+'/custom_data_import_files/'+experiment+'_custom.xlsx')
    custom_data.head()
except:
    custom_data = pd.DataFrame(columns=['Timepoint Sample'])
    
custom_data

,Timepoint Sample


#### 3.2.5 Merging all offline data

In [13]:
ac_merge = pd.merge(left=analytical_data_pivot, right=trace_metal_data_pivot, how='left')
custom_merge = pd.merge(left=ac_merge, right=custom_data, how='left')
offline_data = pd.merge(left=timepoint_data, right=custom_merge, how='left')
offline_data.head()

,Timepoint Sample,Experiment,Parent Culture,Reactor/Plate/Flask Number,Timepoint (#),Timepoint (h),Sample volume (mL),OD,D-glucose [g/L],acetic acid [g/L],citric acid [g/L],ethanol [g/L],formic acid [g/L],lactate [g/L],potassium dihydrogen phosphate [g/L],pyruvic acid [g/L],succinic acid [g/L]
0,DDB_PD_005_AMBR_R01__S00,DDB_PD_005_AMBR,DDB_PD_005_AMBR_DDB7-002_R01_Main_,R01,S00,0.036495,1.0,0.256,21.867193,0.000000,0.0,0.175717,0.0,0.000000,9.618414,0.000000,0.0
1,DDB_PD_005_AMBR_R01__S01,DDB_PD_005_AMBR,DDB_PD_005_AMBR_DDB7-002_R01_Main_,R01,S01,3.047016,1.0,0.628,21.757897,0.000000,0.0,0.141002,0.0,0.000000,11.672140,0.000000,0.0
2,DDB_PD_005_AMBR_R01__S02,DDB_PD_005_AMBR,DDB_PD_005_AMBR_DDB7-002_R01_Main_,R01,S02,6.018670,2.5,1.800,20.801498,0.129068,0.0,0.106812,0.0,0.000000,10.974623,0.000000,0.0
3,DDB_PD_005_AMBR_R01__S03,DDB_PD_005_AMBR,DDB_PD_005_AMBR_DDB7-002_R01_Main_,R01,S03,9.016564,1.0,5.365,16.772116,0.362876,0.0,0.146651,0.0,0.580863,9.833429,0.098438,0.0
4,DDB_PD_005_AMBR_R01__S04,DDB_PD_005_AMBR,DDB_PD_005_AMBR_DDB7-002_R01_Main_,R01,S04,12.019295,1.0,14.650,9.421337,0.568955,0.0,0.191405,0.0,1.082594,9.341226,0.308042,0.0


## 4. Export data

### 4.2 Separate csv files

In [14]:
meta_data.to_csv('../../../data/fermentation_data_exports/' + experiment + '_meta_data_export.csv',index=False)
offline_data.to_csv('../../../data/fermentation_data_exports/' + experiment + '_offline_data_export.csv',index=False)
online_data.to_csv('../../../data/fermentation_data_exports/' + experiment + '_online_data_export.csv',index=False)